Spanish Word Embeddings: [link](https://github.com/dccuchile/spanish-word-embeddings?tab=readme-ov-file)

Word vectors for 157 languages: [link](https://fasttext.cc/docs/en/crawl-vectors.html)

In [ ]:
# Para manipulación de datos y operaciones numéricas
import numpy as np
# Para vectorización y modelos de Machine Learning
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
# Para evaluación
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
# Para visualización (opcional, para la matriz de confusión)
import matplotlib.pyplot as plt

print("Librerías principales importadas.")

In [ ]:
# Demostración: CountVectorizer (Bag-of-Words) vs TfidfVectorizer

# Corpus de ejemplo (textos cortos en español)
corpus = [
    "qué bueno está el mate amargo",
    "el mate dulce no me va",
    "qué amargo está este mate che",
    "me gusta el mate bien caliente"
]

In [ ]:
# 1. Usando CountVectorizer (Bolsa de Palabras simple)
print("--- CountVectorizer (BoW) ---")
count_vectorizer = CountVectorizer()
bow_matrix = count_vectorizer.fit_transform(corpus)

# Mostramos el vocabulario (las "features")
print("Vocabulario (Features):")
print(count_vectorizer.get_feature_names_out())

# Mostramos la matriz BoW (dispersa)
print("\nMatriz BoW (Documento x Palabra):")
print(bow_matrix.toarray()) # .toarray() para verla densa (cuidado con corpus grandes)
print("-" * 30)

In [ ]:
# 2. Usando TfidfVectorizer
print("\n--- TfidfVectorizer ---")
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# El vocabulario es el mismo (si no cambiamos parámetros)
# print("Vocabulario (Features):") # Ya lo mostramos arriba
# print(tfidf_vectorizer.get_feature_names_out())

# Mostramos la matriz TF-IDF
print("Matriz TF-IDF (Documento x Palabra):")
print(tfidf_matrix.toarray())
print("-" * 30)

# --- Observaciones ---
# Nota 1: Compara los valores de palabras comunes como 'mate', 'el', 'está'.
#         En BoW son solo cuentas. En TF-IDF, su peso se ajusta por la frecuencia
#         inversa en los documentos (IDF). Palabras más 'distintivas' de un
#         documento (ej. 'dulce', 'caliente', 'che') tienden a tener mayor peso TF-IDF
#         relativo que palabras muy comunes en *todo* el corpus.
# Nota 2: Ambas matrices son 'dispersas' (muchos ceros).

In [ ]:
# --- Demostración: Añadiendo N-Gramas a TF-IDF ---

print("\n--- TfidfVectorizer con N-Gramas (bigramas) ---")

# Usamos el mismo corpus de antes
# Creamos un nuevo vectorizador, ahora especificando ngram_range
# ngram_range=(1, 2) significa: usa unigramas (palabras solas) Y bigramas (pares de palabras)
tfidf_ngram_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
tfidf_ngram_matrix = tfidf_ngram_vectorizer.fit_transform(corpus)

# Mostramos el NUEVO vocabulario, que ahora incluye bigramas
print("Vocabulario con unigramas y bigramas:")
print(tfidf_ngram_vectorizer.get_feature_names_out())

# Mostramos la nueva matriz TF-IDF (será más ancha)
print("\nMatriz TF-IDF con N-Gramas:")
print(tfidf_ngram_matrix.toarray())
print("-" * 30)

# --- Observaciones ---
# Nota 1: Fíjate cómo aparecen términos como "mate amargo", "qué bueno", "no me", etc.
#         Estos bigramas capturan un poco del contexto local y pueden ser features
#         muy útiles para los modelos. Por ejemplo, 'amargo' solo y 'mate amargo'
#         aportan información diferente.
# Nota 2: El número de features (columnas) aumenta considerablemente.

In [ ]:
# --- Demostración: Pipeline de Clasificación y Evaluación ---

# 1. Datos de Ejemplo (Simples, para clasificación binaria: Positivo/Negativo)
textos = [
    "La milanesa a caballo estaba espectacular!", # Positivo
    "Qué buena onda la atención en el bar.",     # Positivo
    "El flan con dulce de leche es lo más.",    # Positivo
    "El bife de chorizo llegó frío y duro.",     # Negativo
    "Mucho quilombo, tardaron una banda en traer la cuenta.", # Negativo
    "La verdad, la pizza dejaba bastante que deseear.", # Negativo
]
# Etiquetas: 1 para Positivo, 0 para Negativo
labels = np.array([1, 1, 1, 0, 0, 0])

# 2. Dividir datos en Entrenamiento y Prueba
X_train, X_test, y_train, y_test = train_test_split(textos, labels, test_size=0.33, random_state=42)
# test_size=0.33 significa que ~1/3 va para prueba. random_state para reproducibilidad.

print(f"Textos de entrenamiento: {len(X_train)}")
print(f"Textos de prueba: {len(X_test)}")
print("-" * 30)

# 3. Crear y Entrenar un Pipeline (TF-IDF + Naive Bayes)
# El Pipeline encadena pasos: primero vectoriza, luego clasifica.
pipeline_nb = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,1))), # Usamos solo unigramas aquí por simplicidad
    ('clf', MultinomialNB()) # Clasificador Naive Bayes Multinomial
])

print("\nEntrenando el Pipeline (TF-IDF + Naive Bayes)...")
# Entrenamos el pipeline COMPLETO con los datos de texto CRUDOS de entrenamiento
pipeline_nb.fit(X_train, y_train)
print("Entrenamiento completado.")
print("-" * 30)

# 4. Realizar Predicciones sobre los datos de Prueba
print("\nRealizando predicciones sobre el conjunto de prueba...")
y_pred_nb = pipeline_nb.predict(X_test)

print(f"Predicciones: {y_pred_nb}")
print(f"Etiquetas Reales: {y_test}")
print("-" * 30)

# 5. Evaluar el Modelo
print("\n--- Evaluación del Modelo (Naive Bayes) ---")
print(classification_report(y_test, y_pred_nb, target_names=['Negativo (0)', 'Positivo (1)']))

# Matriz de Confusión
print("\nMatriz de Confusión:")
cm_nb = confusion_matrix(y_test, y_pred_nb)
disp_nb = ConfusionMatrixDisplay(confusion_matrix=cm_nb, display_labels=['Negativo (0)', 'Positivo (1)'])
disp_nb.plot(cmap=plt.cm.Blues)
plt.show() # Muestra el gráfico

# --- Observaciones ---
# Nota 1: El Pipeline simplifica enormemente el flujo. Entrenamos con texto crudo,
#         predice con texto crudo. La vectorización ocurre dentro del pipeline.
# Nota 2: El 'classification_report' es CLAVE. Muestra Precision, Recall y F1-Score
#         para cada clase. Fijarse si el modelo funciona igual de bien para ambas.
#         'support' indica cuántas muestras de cada clase había en el test set.
# Nota 3: La Matriz de Confusión ayuda a ver *dónde* se equivoca.
#         Diagonal principal = Aciertos. Fuera de la diagonal = Errores.
#         (Fila = Real, Columna = Predicción)

In [ ]:
!pip install --upgrade numpy scipy gensim

In [ ]:
!pip uninstall gensim -y # Remove the existing gensim installation
!pip install gensim # Reinstall gensim to align with the NumPy version
# Restart the kernel to ensure the changes take effect

In [ ]:
import gensim.downloader as api
from gensim.models import KeyedVectors

In [ ]:
# List available models to find the correct name
print(api.info()['models'].keys())

In [ ]:
model_name = 'fasttext-wiki-news-subwords-300'
word_vectors = api.load(model_name)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print(f"Modelo cargado. Vocabulario: {len(word_vectors.index_to_key)} palabras.")
modelo_cargado = True

In [ ]:
if modelo_cargado:
    # a) Obtener el vector de una palabra
    try:
        vector_auto = word_vectors['auto']
        print(f"Vector para 'auto' (primeros 10 de {len(vector_auto)} dimensiones):")
        print(vector_auto[:10])
        print(f"Forma del vector: {vector_auto.shape}") # Debería ser (300,) si usas SBWC de 300d
    except KeyError:
        print("La palabra 'auto' no está en el vocabulario.")
    print("-" * 30)

In [ ]:
# b) Encontrar palabras similares (similitud coseno)
try:
  similares_mate = word_vectors.most_similar('elefante', topn=10)
  print("Palabras más similares a 'elefante':")
  for palabra, score in similares_mate:
    print(f"- {palabra}: {score:.5f}") # Muestra la palabra y su puntaje de similitud
except KeyError:
  print("La palabra 'elefante' no está en el vocabulario.")
print("-" * 30)

In [ ]:
# c) Realizar analogías vectoriales (el famoso ejemplo rey/reina)
try:
  analogia = word_vectors.most_similar(positive=['casa', 'edificio'], negative=['carpa'], topn=2)
  print("Analogía: rey - varón + mujer ≈ ???")
  print(f"Resultado más probable: {analogia[0][0]} (Score: {analogia[0][1]:.4f})")
except KeyError as e:
  print(f"Error en la analogía: Falta la palabra '{e.args[0]}' en el vocabulario.")
print("-" * 30)

# GUÍA DE ESTUDIO - SÍNTESIS E INTEGRACIÓN

## Preguntas y Respuestas Clave

### **Integración de Técnicas**

**P: ¿Cuál es la ventaja principal de usar Pipeline en scikit-learn?**  
R: Encadena automáticamente pasos (vectorización + clasificación) y evita data leakage aplicando transformaciones solo en datos de entrenamiento durante fit.

**P: ¿Por qué usar train_test_split antes de crear el pipeline?**  
R: Para evaluar objetivamente el rendimiento en datos no vistos y detectar overfitting del modelo completo.

**P: ¿Qué información proporciona classification_report?**  
R: Precision (de predicciones positivas, cuántas son correctas), Recall (de casos reales positivos, cuántos detectamos), y F1-score (balance entre ambas).

### **Comparación de Métodos**

**P: ¿Cuándo añadir n-gramas a TF-IDF?**  
R: Cuando el contexto local es importante ("muy bueno" vs "bueno muy"), pero aumenta dimensionalidad y complejidad.

**P: ¿Cómo comparar objetivamente BoW vs TF-IDF vs Embeddings?**  
R: Usar mismo pipeline, mismos datos de train/test, comparar métricas finales (F1-score, accuracy) en tarea específica.

**P: ¿Por qué la matriz de confusión es importante?**  
R: Muestra patrones de error específicos: ¿confunde más falsos positivos o falsos negativos? ¿Qué clases se confunden entre sí?

### **Embeddings en Pipelines**

**P: ¿Cómo integrar word embeddings en un pipeline de clasificación?**  
R: Promediando vectores de palabras del documento o usando representaciones más sofisticadas como weighted averages por TF-IDF.

**P: ¿Qué ventajas tienen embeddings sobre TF-IDF en clasificación?**  
R: Capturan semántica (sinónimos contribuyen similarmente), menor dimensionalidad, mejor generalización a vocabulario no visto.

### **Decisiones Prácticas**

**P: ¿Cómo decidir entre Naive Bayes y Logistic Regression?**  
R: NB asume independencia de features, rápido, bueno para texto. LR más flexible, maneja correlaciones, mejor con features continuas.

**P: ¿Cuándo usar cada método en producción?**  
R: BoW/TF-IDF para baseline rápido, FastText para robustez OOV, Word2Vec para análisis semántico, según requirements específicos.

### **Evaluación Integral**

**P: ¿Qué métricas usar para comparar sistemas de NLP?**  
R: Accuracy (básico), F1-score (balanceado), Precision/Recall específicos según costo de errores, tiempo de inferencia.

**P: ¿Cómo detectar overfitting en NLP?**  
R: Gran diferencia entre performance train/validation, alta dimensionalidad vs pocos datos, memorización de vocabulario específico.

## Puntos Clave para Recordar

1. **Pipeline automatiza y previene errores** en flujo ML
2. **Evaluación objetiva require train/test split** apropiado
3. **Diferentes métodos para diferentes problemas** - no hay silver bullet
4. **Métricas específicas revelan fortalezas/debilidades** de cada approach
5. **Embeddings aportan semántica** pero requieren más setup
6. **Integración práctica** considera tiempo, memoria, interpretabilidad

## Errores Comunes a Evitar

- Aplicar transformaciones a todo el dataset antes del split
- Comparar métodos con diferentes preprocessings
- Elegir solo por accuracy sin considerar precision/recall
- No validar assumptions de algoritmos (ej: independencia en NB)
- Ignorar computational requirements en producción

## Conexión con Próxima Clase

Esta integración de métodos tradicionales y modernos prepara para **aplicaciones avanzadas**: extracción de información, análisis de entidades, y sistemas de NLP end-to-end.

---
*Consejo: Siempre establece baseline simple (BoW + NB) antes de probar métodos complejos. Te da referencia para validar si la complejidad adicional vale la pena.*